In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
import numpy as np
import imageio
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
import cv2
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
import random
random.seed(2) # Python
np.random.seed(1) #numpy
from tensorflow import set_random_seed
set_random_seed(3) # Tensorflow

In [0]:
#The path to the omniglot data
root_path = "./drive/My Drive/"
train_path = os.path.join(root_path,'training') 
validation_path = os.path.join(root_path,'validation')

def load_image(path, n = 0):
    X = []
    
    #Load every srud seperately and place that in one tensor
    for student in os.listdir(path):
#         print("Loading student: " + student)
        student_path = os.path.join(path, student)
       
        category_images = []
            
        if not os.path.isdir(student_path):
                continue
            
            #Read evey image with in the directory
        for filename in os.listdir(student_path):
                image_path = os.path.join(student_path, filename)
                image = imageio.imread(image_path)
                width = 105
                height = 105 # keep original height
                dim = (height,width)
 
                # resize image
                image = cv2.resize(image, dim)
                image=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                image= np.expand_dims(image, axis=0)
#                 print(image_path)
                #Image preprocessing
                image = image/255
                image = 1 - image
                
                X.append(image)
                
        
    X = np.stack(X)
    return X

In [5]:
print("Loading Training Set")
Xtrain = load_image(train_path)
print(Xtrain.shape)

Loading Training Set
(1470, 1, 105, 105)


In [6]:
print("Now loading evaluation set")
Xval = load_image(validation_path)
print(Xval.shape)

Now loading evaluation set
(646, 1, 105, 105)


In [0]:
def get_batch(data, batch_size):
    n_classes, n_examples, h, w = data.shape
    
    pairs = [np.zeros((batch_size, 1, h, w)) for i in range(2)]
    
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    
    categories = np.random.choice(n_classes, size = (batch_size), replace = False)
    
    for i in range(batch_size):
        category = categories[i]
        
        idx1 = np.random.randint(0, n_examples)
        pairs[0][i,:,:,:] = data[category, idx1].reshape(1, h,w)
        idx2 = np.random.randint(0, n_examples)
        
        if targets[i] == 0:
            category_2 = category
        else:
            category_2 = (category + np.random.randint(1, n_classes)) % n_classes
 
        
        pairs[1][i,:,:,:] = data[category_2, idx2].reshape(1, h, w)
        
    return pairs, targets
        



In [0]:
def generate(data,batch_size):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = get_batch(Xtrain,batch_size)
            yield (pairs, targets)    


 

In [0]:
def make_oneshot_task(N,data,language=None):
        """Create pairs of test image, support set for testing N way one-shot learning. """
        n_classes, n_examples, w, h = data.shape
        indices = np.random.randint(0,n_examples,size=(N,))
        categories = np.random.choice(range(n_classes),size=(N,),replace=False)            
        true_category = categories[0]
        ex1 = np.random.randint(0, n_examples)
        ex2 = np.random.randint(0, n_examples)
        # ex1, ex2 = np.random.choice(n_examples,replace=False,size=(0,))
        test_image = np.asarray([data[true_category,ex1,:,:]]*N).reshape(N, 1, w,h)
        support_set = data[categories,indices,:,:]
        support_set[0,:,:] = data[true_category,ex2]
        support_set = support_set.reshape(N, 1, w,h)
        targets = np.zeros((N,))
        targets[0] = 1
        targets, test_image, support_set = shuffle(targets, test_image, support_set)
        pairs = [test_image,support_set]

        return pairs, targets
    

                   

In [0]:
def test_oneshot(model,N,k,data,verbose=0):
        """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
        n_correct = 0
        if verbose:
            print("Evaluating model on {} random {} way one-shot learning tasks ...".format(k,N))
        for i in range(k):
            inputs, targets = make_oneshot_task(N,data)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct
    


In [0]:
def train(model, epochs, verbosity):
        model.fit_generator(self.generate(batch_size),)

In [0]:
from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input, Lambda
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
import time

In [0]:
#Contrastive Loss
def euclid_dist(input_pair):
    x, y = input_pair
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    return distance

def euclid_dist_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = -1 * y_true + 1
    return K.mean((1-y_true) * K.square(y_pred) + y_true *  K.square(K.maximum(margin - y_pred, 0.0)))

def acc(y_true, y_pred):
    ones = K.ones_like(y_pred)
    return K.mean(K.equal(y_true, ones - K.clip(K.round(y_pred), 0, 1)), axis=-1)



In [0]:
INIT_WEIGHTS = os.path.join(root_path, 'init_weights.hdf5')
CHECKPOINTED_WEIGHTS = os.path.join(root_path, 'checkpointed_weights.hdf5')

In [0]:
im_width  =105
im_height =105
im_chan   =1      

In [16]:
input_img =Input(( im_chan,im_height,im_width ), name='img')

# print(input_img)
# Down Block 1
c1 = Conv2D(32,(1,1), activation='relu', kernel_regularizer=l2(2e-4) )(input_img)
p1 = MaxPooling2D(data_format="channels_first") (c1)

#Down Block 2
c2 = Conv2D(128, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p1)
p2 = MaxPooling2D(data_format="channels_first") (c2)

#Down Block 3
c3 = Conv2D(128, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p2)
p3 = MaxPooling2D(data_format="channels_first") (c3)

#Down Block 4
c4 = Conv2D(256, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p3)
p4 = MaxPooling2D(data_format="channels_first") (c4)
# # print(p4)
flat = Flatten()(p4)
output = Dense(1, activation='sigmoid')(flat)

# Instantiate the Model
model = Model(input_img, output)

input1 = Input(shape = (1,105,105))
input2 = Input(shape = (1,105,105))

output1 = model(input1)
output2 = model(input2)


distance = Lambda(euclid_dist, output_shape=euclid_dist_shape)([output1, output2])

prediction = Dense(1, activation='sigmoid')(distance)
    
siamese = Model(inputs=[input1, input2], outputs=prediction)
    
optimizer = Adam(lr = 0.00006)
    
siamese.compile(loss=contrastive_loss, optimizer=optimizer, metrics=[acc])

siamese.summary()
os.system("rm {}".format(INIT_WEIGHTS))
siamese.save_weights(INIT_WEIGHTS)












Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 105, 105)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 105, 105)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            31297       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]   

In [20]:
#Training loop
print("!")
evaluate_every = 1 # interval for evaluating on one-shot tasks
loss_every=50 # interval for printing loss (iterations)
batch_size = 32
n_iter = 30
N_way = 50 # how many classes for testing one-shot tasks>
n_val = 30 #how mahy one-shot tasks to validate on?
best = -1
data_path = "./drive/My Drive/Colab Notebooks"
weights_path= CHECKPOINTED_WEIGHTS
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter):
    (inputs,targets)=get_batch(Xtrain,batch_size)
    loss=siamese.train_on_batch(inputs,targets)
    print(loss)
    if i % evaluate_every == 0:
        print("Time for {0} iterations: {1}".format(i, time.time()-t_start))
        val_acc = test_oneshot(siamese,N_way,n_val,Xval,verbose=True)
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            print("Saving weights to: {0} \n".format(weights_path))
            siamese.save(weights_path)
            best=val_acc

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))
# weights_path_2 = os.path.join(data_path, "weight.h5")
# siamese.load_weights(CHECKPOINTED_WEIGHTS)

!
Starting training process!
-------------------------------------
[0.29838386, 0.5]
Time for 1 iterations: 0.06723451614379883
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 70.0% 50 way one-shot learning accuracy
Current best: 70.0, previous best: -1
Saving weights to: ./drive/My Drive/checkpointed_weights_omniglot.hdf5 

[0.29799283, 0.5]
Time for 2 iterations: 1.751222848892212
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 70.0% 50 way one-shot learning accuracy
Current best: 70.0, previous best: 70.0
Saving weights to: ./drive/My Drive/checkpointed_weights_omniglot.hdf5 

[0.298381, 0.5]
Time for 3 iterations: 3.259033679962158
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 80.0% 50 way one-shot learning accuracy
Current best: 80.0, previous best: 70.0
Saving weights to: ./drive/My Drive/checkpointed_weights_omniglot.hdf5 

[0.29902518, 0.5]
Time for 4 iterations: 4.7513430

In [21]:
print(inputs[1].shape)
# plot_oneshot_task(inputs)
p=siamese.predict(inputs)
print(p)

(32, 1, 105, 105)
[[0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.5003297 ]
 [0.4750408 ]
 [0.47657776]
 [0.4794998 ]
 [0.48850816]
 [0.49278167]
 [0.48520684]
 [0.46882784]
 [0.4732679 ]
 [0.4939587 ]
 [0.4564135 ]
 [0.4640463 ]
 [0.491099  ]
 [0.47420925]
 [0.48517922]
 [0.48425993]
 [0.49493802]]
